Just importing stuff

In [2]:
%pip install keras
%pip install PyQt5
import pandas as pd
import numpy as np
import tensorflow as tf
import PyQt5
import matplotlib.pyplot as plt
import datetime as dt



Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'e:\Python\python3.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'e:\Python\python3.exe -m pip install --upgrade pip' command.


In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM
from tensorflow.keras.optimizers import Adam

In [4]:
data = pd.read_csv("../complete_merged.csv",index_col=0)

Consecutive date checking function (Some districts had partial data)

Split Sequence(Standard function for splitting data for univariate lstm)

In [32]:
#Testing datetime consecutiveness


def checkConsecutiveDates(date_strs):
    dates = [dt.datetime.strptime(d, "%Y-%m-%d") for d in date_strs]

    date_ints = set([d.toordinal() for d in dates])
    if len(date_ints) == 1:
        return True
    elif max(date_ints) - min(date_ints) == len(date_ints) - 1:
        return True
    else:
        return False
    


def split_sequence(date_sequence,temp_sequence, n_steps,n_features):
    X, y = list(), list()
    for i in range(len(temp_sequence)):
    # find the end of this pattern
        end_ix = i + n_steps
 # check if we are beyond the sequence
        if end_ix > len(temp_sequence)-1:
            break
 # gather input and output parts of the pattern
        seq_x, seq_y = temp_sequence[i:end_ix], temp_sequence[end_ix]
        date_seq = date_sequence[i:end_ix]
        # print(checkConsecutive(seq_x))
        if(checkConsecutiveDates(date_seq)):
            X.append(seq_x)
            y.append(seq_y)
    X,y = np.array(X), np.array(y)
    X = X.reshape((X.shape[0], X.shape[1], n_features))
        
    return X,y

def get_data(district,column):
    return data[data['district'] == district].sort_values(axis = 0,by = "date")[column].to_list()
   



# date_strs = ['02-29-2012', '02-28-2012', '03-01-2012']
date_strs = ['1999-12-31','2000-01-01']
print(checkConsecutiveDates(date_strs))

X,y = split_sequence(['2021-01-09','2021-01-11','2021-01-12','2021-01-13','2021-01-14'],[10, 20, 30, 40, 50],3,1)
for i in range(len(X)):
    print(X[i], y[i])





True
[[20]
 [30]
 [40]] 50


Function to evaluate model(Mean Square Error)

In [13]:
def evalLSTM(model,X,y,n_steps,n_features):
    y_predict = list()
    for i in range(X.shape[0]):
        x_input = X[i]
        x_input = x_input.reshape((1, n_steps, n_features))
        predicted_value = model.predict(x_input, verbose=0)[0][0]
        print(len(y_predict),"of",X.shape[0])
        y_predict.append(predicted_value)
    
    mse = tf.keras.losses.MeanSquaredError()
    
    return {'mse':mse(y_predict,y).numpy(),'y_predict':y_predict}


Defining the model

In [14]:
#different optimizers and loss functions need to be tested yet
n_steps = 10
n_features = 1
epochs =20
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')



Basic check for Adilabad

In [15]:
%%script false 

data_Adilabad_temp = get_data('Adilabad','temp_max')
data_Adilabad_date = get_data('Adilabad','date')
# print(data_Adilabad_date)
X,y = split_sequence(data_Adilabad_date,data_Adilabad_temp,n_steps,n_features)

model.fit(X, y, epochs=200, verbose=4)
mse = evalLSTM(model,X,y,10,1)

mse
#3.36 for 200 epochs


Couldn't find program: 'false'


In [39]:
%%script false 
date_X,date_y = split_sequence(data_Adilabad_date,data_Adilabad_date,n_steps,n_features)

%matplotlib qt
date_y = np.asarray(date_y, dtype='datetime64[s]')

plt.plot(date_y,y)
plt.plot(date_y,mse['y_predict'])
plt.show()


Various window sizes were used on Adilabad for optimization.Results were as follows:

[10-3.289,
 20-55142.400,
 30-6766769.447,
 40-216242835.855,
 50-14841958357.522,
 60-793333303283.484]


[7-116.768,
 8-28.928,
 9-5.358,
 10-3.289,
 11-5.317]

Concluding that 10 is an optimal window size

Training on complete data of all districts and multiple window sizes


In [ ]:

%%script false


# steps = [5,10,20,30,60,90]
steps = [90]
for i in steps:
    n_steps = i
    n_features = 1
    epochs =20
    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')

    
    for district in data['district'].unique():
        print(str(i) + "out of "+str(len(data['district'].unique())))
        
        data_unique_district_temp = get_data(district,'temp_max')
        data_unique_district_date = get_data(district,'date')
        
        print(district,str(len(data_unique_district_temp)))
        X,y = split_sequence(data_unique_district_date, data_unique_district_temp,n_steps,n_features)
        model.fit(X, y, epochs=epochs, verbose=0)
        

    model.save('saved_model/model_e20_w'+str(n_steps))

Couldn't find program: 'false'


Loading a model


In [16]:
model = tf.keras.models.load_model('saved_model/model_e20_w5')
n_steps = 5
n_features = 1


Comparing true and pred for adilabad

In [17]:
data_unique_district_temp = get_data('Adilabad','temp_max')
data_unique_district_date = get_data('Adilabad','date')
    
  
X,y = split_sequence(data_unique_district_date, data_unique_district_temp,n_steps,n_features)
metrics = evalLSTM(model,X,y,5,1)


0 of 1724
1 of 1724
2 of 1724
3 of 1724
4 of 1724
5 of 1724
6 of 1724
7 of 1724
8 of 1724
9 of 1724
10 of 1724
11 of 1724
12 of 1724
13 of 1724
14 of 1724
15 of 1724
16 of 1724
17 of 1724
18 of 1724
19 of 1724
20 of 1724
21 of 1724
22 of 1724
23 of 1724
24 of 1724
25 of 1724
26 of 1724
27 of 1724
28 of 1724
29 of 1724
30 of 1724
31 of 1724
32 of 1724
33 of 1724
34 of 1724
35 of 1724
36 of 1724
37 of 1724
38 of 1724
39 of 1724
40 of 1724
41 of 1724
42 of 1724
43 of 1724
44 of 1724
45 of 1724
46 of 1724
47 of 1724
48 of 1724
49 of 1724
50 of 1724
51 of 1724
52 of 1724
53 of 1724
54 of 1724
55 of 1724
56 of 1724
57 of 1724
58 of 1724
59 of 1724
60 of 1724
61 of 1724
62 of 1724
63 of 1724
64 of 1724
65 of 1724
66 of 1724
67 of 1724
68 of 1724
69 of 1724
70 of 1724
71 of 1724
72 of 1724
73 of 1724
74 of 1724
75 of 1724
76 of 1724
77 of 1724
78 of 1724
79 of 1724
80 of 1724
81 of 1724
82 of 1724
83 of 1724
84 of 1724
85 of 1724
86 of 1724
87 of 1724
88 of 1724
89 of 1724
90 of 1724
91 of 172

In [18]:
date_X,date_y = split_sequence(data_unique_district_date,data_unique_district_date,n_steps,n_features)

%matplotlib qt
date_y = np.asarray(date_y, dtype='datetime64[s]')

plt.plot(date_y,y)
plt.plot(date_y,metrics['y_predict'])
plt.show()

In [19]:
%%script false

temp_2023 = np.array([29.0,28.0,27.0,21.0,19.0,26.0,27.0,28.0,30.0,31.0])
temp_window = temp_2023[-10:]
for i in range(100):
    temp_window = temp_window.reshape(1, n_steps, n_features)
    predicted_value = model.predict(temp_window, verbose=0)[0][0]
    print(predicted_value)
    temp_2023 = np.append(temp_2023, predicted_value)
    temp_window = temp_2023[-10:]


Couldn't find program: 'false'


Trying to create a csv file(cell in progress)


In [42]:
%%script false

final_df = pd.DataFrame()



i = 1
# for district in data['district'].unique():
for district in ['Adilabad']:
    temp_df = pd.DataFrame()
    print(str(i) + "out of "+str(len(data['district'].unique())))
    
    data_unique_district_temp = get_data(district,'temp_max')
    data_unique_district_date = get_data(district,'date')
   
    print(district,str(len(data_unique_district_temp)))
    X,y = split_sequence(data_unique_district_date, data_unique_district_temp,n_steps,n_features)
    date_X,date_y = split_sequence(data_unique_district_date,data_unique_district_date,n_steps,n_features)
    metrics = evalLSTM(model,X,y,10,1)

    temp_df['date'] = date_y
    temp_df['predict_temp_max'] = metrics['y_predict']
    temp_df['district'] = district
    temp_df['window_size'] = 10

    final_df.concat(temp_df)
    
    
    i = i+1

temp_df

1out of 46
Adilabad 1733
0 of 1714
1 of 1714
2 of 1714
3 of 1714
4 of 1714
5 of 1714
6 of 1714
7 of 1714
8 of 1714
9 of 1714
10 of 1714
11 of 1714
12 of 1714
13 of 1714
14 of 1714
15 of 1714
16 of 1714
17 of 1714
18 of 1714
19 of 1714
20 of 1714
21 of 1714
22 of 1714
23 of 1714
24 of 1714
25 of 1714
26 of 1714
27 of 1714
28 of 1714
29 of 1714
30 of 1714
31 of 1714
32 of 1714
33 of 1714
34 of 1714
35 of 1714
36 of 1714
37 of 1714
38 of 1714
39 of 1714
40 of 1714
41 of 1714
42 of 1714
43 of 1714
44 of 1714
45 of 1714
46 of 1714
47 of 1714
48 of 1714
49 of 1714
50 of 1714
51 of 1714
52 of 1714
53 of 1714
54 of 1714
55 of 1714
56 of 1714
57 of 1714
58 of 1714
59 of 1714
60 of 1714
61 of 1714
62 of 1714
63 of 1714
64 of 1714
65 of 1714
66 of 1714
67 of 1714
68 of 1714
69 of 1714
70 of 1714
71 of 1714
72 of 1714
73 of 1714
74 of 1714
75 of 1714
76 of 1714
77 of 1714
78 of 1714
79 of 1714
80 of 1714
81 of 1714
82 of 1714
83 of 1714
84 of 1714
85 of 1714
86 of 1714
87 of 1714
88 of 1714
89 of 

,Adilabad,date
0,32.905209,2018-01-11
1,29.994761,2018-01-12
2,31.974672,2018-01-13
3,32.406506,2018-01-14
4,33.111984,2018-01-15
...,...,...
1709,32.657871,2022-09-26
1710,33.213398,2022-09-27
1711,32.543259,2022-09-28
1712,33.276501,2022-09-29
